In [1]:
import urllib2
from bs4 import BeautifulSoup
from pprint import pprint
from datetime import datetime

In [4]:
import pymongo

from pymongo import MongoClient 
c = MongoClient() 
c['espace5'].drop_collection('forumsbis')

db = c['espace5']
forum_messages = db['forumsbis']

In [5]:
data = []
url = "http://www.forum-auto.com/marques/renault/sujet5226-"

for scan in range(3395-1260,3046,35):
    scan_page = url+str(scan+1260)+'.htm'
    page = urllib2.urlopen(scan_page)
    soup = BeautifulSoup(page)
    case1 = soup.findAll("td", { "class" : "messCase1" })
    case2 = soup.findAll("td", { "class" : "messCase2" })
    for x,y in zip(case1, case2):
        try:
            user_id = x.find("span").text
            date_posted = y.find("div", { "class" : "toolbar" }).text
            date_posted = datetime.strptime(date_posted[9:19]+" "+date_posted[22:30], '%d-%m-%Y %H:%M:%S')
            try:
                y.find("div", { "class" : "toolbar" }).extract()
            except:
                pass
            try:
                y.find("table", { "class" : "citation" }).extract()
            except:
                pass
            try:
                y.find("div", { "class" : "edited" }).extract()
            except:
                pass
            try:
                y.find("div", { "class" : "edited" }).extract()
            except:
                pass
            try:
                y.find("span", { "class" : "signature" }).extract()
            except:
                pass
            try:
                photos = y.findAll("span", { "class" : "bbcode display_photo width_650" })
                for photo in photos:
                    photo.extract()
            except:
                pass

            doc = {'text': y.getText(" "), 'user': user_id, 'date': date_posted , 'forum_id' : 'forum-auto-secrets-espace-5'}
            forum_messages.save(doc)
        except:
            pass

In [6]:
forum_messages.count()

964

In [7]:
m = forum_messages.find_one()
print m['text']


 Reprise du message précédent : Merci de l'info! 


In [8]:
url = "http://www.planeterenault.com/forum/espace-v-topic-unique-vos-avis-infos-impressions-sur-la-5eme-generation-d-espace-t45815-"

for scan in range(0,109,18):
    scan_page = url+str(scan)+'.html'
    page = urllib2.urlopen(scan_page)
    soup = BeautifulSoup(page)
    rows = soup.findAll("table", { "class" : "row1" })
    rows += soup.findAll("table", { "class" : "row2" })
    for row in rows:
        user = row.find("td", { "class" : "user" })
        user_id = user.find("span", { "class" : "auteurPost" }).text.strip(' \t\n\r').rstrip(' \t\n\r')
        user.extract()
        message_id = row.find("a")['href']
        message_id = message_id[1+message_id.index('#'):]
        date_raw = row.find("a").parent
        date_posted = date_raw.text.lstrip(' \t\n\r').rstrip(' \t\n\r')
        date_posted = date_posted[4:8]+date_posted[11:]
        mapping = { 'S':'09', 'O':'10', 'N':'11', 'D':'12', 'J':'01', 'F':'02',  'M':'03',  'A':'04'}
        for k, v in mapping.iteritems():
            date_posted = date_posted.replace(k, v)
        date_posted = datetime.strptime(date_posted, '%d %m %Y %H:%M:%S')
        date_raw.extract()
        message_raw = row.find("div", { "id" : "message_"+message_id })
        try:
            message_raw.find("table").extract()
        except:
            pass
        message = message_raw.getText(" ").lstrip(' \t\n\r').rstrip(' \t\n\r')

        doc = {'text': message, 'user': user_id, 'date': date_posted , 'forum_id' : 'planeterenault-espace-5'}
        forum_messages.save(doc)

In [9]:
forum_messages.count()

1085

In [10]:
m = forum_messages.find_one({'forum_id' : 'planeterenault-espace-5'})
print m['text']

Hello,
 

 
J'inaugure le sujet général sur l'Espace 5 avec les premičres informations commerciales communiquées par Renault :


In [11]:
print forum_messages.find({'forum_id' : 'forum-auto-secrets-espace-5'}).count()
print forum_messages.find({'forum_id' : 'planeterenault-espace-5'}).count()

964
121
